<a href="https://colab.research.google.com/github/rishabhprashr/Coursera_Capstone/blob/master/battle_of_neighborhoods5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
USA is a large and ethnically diverse country. Its largest city New York has a long history of international immigration. Asians, Europeans,Africans,etc make up a large amount of this population and New York is at the centre of it. Cultural diversity brings along a difference of preferences ,opinions,background and tastes. Our goal here is ananlyzing the diversity and popularity of restaurants in the neighborhoods of New York to come up with a business strategy and the most suitable neighborhood to establish the business among all the given neighborhoods. We will use Foursquare places API to find out venues and explore the neighborhoods and scrape out details in a given radius. Our goal here is finding the popular cuisines of different neigborhoods to establish a fusion style restaurant catering to different tastes of multi ethnic population in a diverse neighborhood.

#Data
We will require the neighborhood data of New York along with their location in latitudes and longitudes.The Neighborhoods in NY has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

We will use this freely availabel dataset on the internet: https://geo.nyu.edu/catalog/nyu_2451_34572

The dataset contains details of location of different neighborhoods under the given boroughs in json format. We transform the json to a pandas dataframe. We now have a pandas dataframe containing the location data of all the neighborhoods. We define a Foursquare API URI containing the 
credentials to make requests for the given neighborhoods using the locations of the given neighborhoods. We define and transform the dataframes to select the features of our requirements. We find the top 3 popular cuisines of each neighborhoods and then combine the dataset to find the most popular cuisines overall for a fusion restaurant.

For example while making an API call to explore the surroundings of a given neighborhood,we will collect the top venues and segregate them based upon the cuisines in a given radius around the neighborhood and make a similar dataset for all the neighborhoods.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print('Matplotlib version: ', mpl.__version__) # >= 2.0.0

print('Libraries imported.')

Matplotlib version:  3.2.1
Libraries imported.


In [0]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)


In [0]:
neighborhoods_data = newyork_data['features']


In [4]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)
neighborhoods['Borough']

The dataframe has 5 boroughs and 306 neighborhoods.


0              Bronx
1              Bronx
2              Bronx
3              Bronx
4              Bronx
5              Bronx
6          Manhattan
7              Bronx
8              Bronx
9              Bronx
10             Bronx
11             Bronx
12             Bronx
13             Bronx
14             Bronx
15             Bronx
16             Bronx
17             Bronx
18             Bronx
19             Bronx
20             Bronx
21             Bronx
22             Bronx
23             Bronx
24             Bronx
25             Bronx
26             Bronx
27             Bronx
28             Bronx
29             Bronx
30             Bronx
31             Bronx
32             Bronx
33             Bronx
34             Bronx
35             Bronx
36             Bronx
37             Bronx
38             Bronx
39             Bronx
40             Bronx
41             Bronx
42             Bronx
43             Bronx
44             Bronx
45             Bronx
46          Brooklyn
47          B

In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [9]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
 
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork) 
    
map_newyork

In [10]:
CLIENT_ID = 'GUAY1GCI335YAPNHNMK5U3CXNCUM1JWM3W4OT1GL3EYQM5KB' # your Foursquare ID
CLIENT_SECRET = 'YVIS5XX2IWORL2FMQ2M3QZV5VZKN2PCRCVESRYJGJSY13LH1' # your Foursquare Secret
VERSION = '20200609' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GUAY1GCI335YAPNHNMK5U3CXNCUM1JWM3W4OT1GL3EYQM5KB
CLIENT_SECRET:YVIS5XX2IWORL2FMQ2M3QZV5VZKN2PCRCVESRYJGJSY13LH1


In [11]:
LIMIT = 200 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id=GUAY1GCI335YAPNHNMK5U3CXNCUM1JWM3W4OT1GL3EYQM5KB&client_secret=YVIS5XX2IWORL2FMQ2M3QZV5VZKN2PCRCVESRYJGJSY13LH1&v=20200609&ll=40.7127281,-74.0060152&radius=1000&limit=200'

In [0]:
results = requests.get(url).json()


In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
results

{'meta': {'code': 429,
  'errorDetail': 'Quota exceeded',
  'errorType': 'quota_exceeded',
  'requestId': '5edfbce740a7ea001beb82db'},
 'response': {}}

In [0]:
def get_category_id(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['id']

In [16]:
venues = results['response']['venues']
k=json_normalize(venues)
filt_columns = ['id','name', 'categories', 'location.lat', 'location.lng']
k_venues =k.loc[:, filt_columns]
k_venues['place categories'] = k_venues.apply(get_category_id, axis=1)
k_venues['categories'] = k_venues.apply(get_category_type, axis=1)
k_venues.columns = [col.split(".")[-1] for col in k_venues.columns]

k_venues



KeyError: ignored

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
manhattan_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [0]:
ny_venues=manhattan_venues
ny_venues.shape

In [0]:
#grouping based on neighborhood to find the top diverse neighborhoods based on food category

top5=ny_venues.groupby('Neighborhood')['Venue Category'].nunique().reset_index()
top5.sort_values(['Venue Category'], ascending=False, axis=0, inplace=True)
top5=top5.head().reset_index(drop=True)
top5.head()






In [0]:

sns.barplot(x="Neighborhood", y="Venue Category", data=top5)

In [0]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

In [0]:
ny_venues.head()

In [0]:
ny_onehot.shape

In [0]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped

In [0]:
ny_grouped.shape

In [0]:
num_top_venues = 5

for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ny_venues_sorted = pd.DataFrame(columns=columns)
ny_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    ny_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

ny_venues_sorted.head()

In [0]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [0]:
# add clustering labels
ny_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(ny_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head() # check the last columns!

In [0]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
rainbow